In [1]:
import cv2
import numpy as np
import dlib

In [2]:
import math
def calculateDistance(x1,y1,x2,y2):
    dist = math.sqrt((x2 - x1)**2 + (y2 - y1)**2)
    return dist

In [3]:
camera=cv2.VideoCapture(1)

detector=dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")
font=cv2.FONT_HERSHEY_COMPLEX
counter=0

while True:
    left_eyes=[]
    right_eyes=[]
    cal_blink_pos=[]
    rec_blink_pos=[]
    rec_garbage_pos=[36,38,39,41,42,44,45,47]
    cal_garbage_pos=[36,39,42,45]
    
    ret, frame=camera.read()
    frame=cv2.flip(frame,1)
    
    gray=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    
    faces=detector(gray)
    for face in faces:        
        landmarks = predictor(gray, face)
    
        for n in range(36, 42):
            x = landmarks.part(n).x
            y = landmarks.part(n).y
            right_eyes.append((x,y))
            if len(right_eyes)>=2:
                pts=np.array(right_eyes, np.int32)
                cv2.fillPoly(frame,[pts], color=(255,255,255))
                cv2.polylines(frame,[pts],True,(255,0,0),2)
        
        for n in range(42, 48):
            x = landmarks.part(n).x
            y = landmarks.part(n).y
            left_eyes.append((x,y))
            if len(left_eyes)>=2:
                pts=np.array(left_eyes, np.int32)
                cv2.fillPoly(frame,[pts], color=(255,255,255))
                cv2.polylines(frame,[pts],True,(255,0,0),2)
        
        
        for n in range(36, 48):
            if n in cal_garbage_pos:
                continue
            else:
                x = landmarks.part(n).x
                y = landmarks.part(n).y
                cal_blink_pos.append((x,y))
                cv2.circle(frame, (x, y), 4, (255, 0, 0), -1)
                if len(cal_blink_pos)>=8:
                    pts=np.array(cal_blink_pos, np.int32)
                    
                    # Left Eyes
                    point_37=cal_blink_pos[0]
                    point_38=cal_blink_pos[1]
                    point_40=cal_blink_pos[2]
                    point_41=cal_blink_pos[3]            

                    dist_37_38=calculateDistance(point_37[0],point_37[1],point_38[0],point_38[1])
                    dist_38_40=calculateDistance(point_38[0],point_38[1],point_40[0],point_40[1])
                    dist_40_41=calculateDistance(point_40[0],point_40[1],point_41[0],point_41[1])
                    dist_41_37=calculateDistance(point_41[0],point_41[1],point_37[0],point_37[1])
            
                    perimeter=dist_37_38+dist_38_40+dist_40_41+dist_41_37
                    cv2.putText(frame,"Perimeter ="+str(perimeter),(400,440),font,0.5,(0,255,0),2)

                    if perimeter>=40:
                        cv2.circle(frame,(20,455),10,(0,255,0),-1)

                    else:
                        counter+=1
                        print(counter)
                            
        
        for n in range(36, 48):
            if n in rec_garbage_pos:
                continue
            else:
                x = landmarks.part(n).x
                y = landmarks.part(n).y
                rec_blink_pos.append((x,y))
                #cv2.circle(panel, (x, y), 4, (255, 0, 0), -1)
                if len(rec_blink_pos)>=4:
                    pts=np.array(rec_blink_pos, np.int32)
                    cv2.rectangle(frame,(rec_blink_pos[0][0],rec_blink_pos[0][1])
                                  ,(rec_blink_pos[1][0],rec_blink_pos[1][1]),(0,0,255),2)
                    cv2.rectangle(frame,(rec_blink_pos[2][0],rec_blink_pos[2][1])
                                  ,(rec_blink_pos[3][0],rec_blink_pos[3][1]),(0,0,255),2)
                    
            
    cv2.imshow('facial landmark',frame)
    #cv2.imshow('output landmark',panel)
    k=cv2.waitKey(1)
    if k==ord('q'):
        break

camera.release()
cv2.destroyAllWindows()

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
